In [1]:
import geopandas as gpd
import pandas as pd
import os
import datetime

# Define paths
data = './data'

# Data Preprocessing

## Geographical units

In [2]:
# Population data obtained from American Community Survey (ACS)
acs_pop = pd.read_csv(os.path.join(data, 'original_data/demand_related/ACSDT5Y2019.B01003_2022-02-21T221149/ACSDT5Y2019.B01003_data_with_overlays_2022-02-21T221145.csv'))
acs_pop = acs_pop[1:]
acs_pop['County'] = acs_pop['GEO_ID'].apply(lambda x:x[9:14])
acs_pop['Tract'] = acs_pop['GEO_ID'].apply(lambda x:x[9:])
acs_pop = acs_pop.rename(columns={'B01003_001E': 'pop'})
acs_pop.drop(columns=['B01003_001M', 'NAME'], inplace=True)
acs_pop['pop'] = acs_pop['pop'].astype(int)
acs_pop.head(3)

,pop,GEO_ID,County,Tract
1,4844,1400000US48001950100,48001,48001950100
2,4838,1400000US48001950401,48001,48001950401
3,7511,1400000US48001950402,48001,48001950402


In [3]:
# Lookup table of TSA for each county
tsa = pd.read_json(os.path.join(data, 'original_data/tsa_county.json'))
tsa['FIPS'] = tsa['FIPS'].astype(str)
tsa.head(3)

,FIPS,TSA,County
0,48011,A,Armstrong
1,48045,A,Briscoe
2,48065,A,Carson


In [4]:
# Calculate population, TSA of each county
county = gpd.read_file(os.path.join(data, 'original_data/demand_related/tl_2021_48_county/tl_2021_48_county.shp'))
county = county[['GEOID', 'NAME', 'geometry']]
county = county.merge(tsa, left_on='GEOID', right_on='FIPS')
county = county.drop(columns=['FIPS', 'NAME'])
county = county.to_crs(epsg=3081)

county_pop = acs_pop.groupby(by='County').sum()
county_pop.reset_index(inplace=True)

county = county.merge(county_pop, left_on='GEOID', right_on='County')
county['County'] = county['County_x']
county.drop(columns=['County_x', 'County_y'], inplace=True)
county.sort_values(by='GEOID', inplace=True, ignore_index=True)
county.to_file(os.path.join(data, 'reference_data', 'geographic_units', 'county_reference.shp'))
county.head(3)

/Users/jparkgeo/opt/anaconda3/envs/sa/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,GEOID,geometry,TSA,pop,County
0,48001,"POLYGON ((1440392.625 1070275.995, 1440394.136...",G,57810,Anderson
1,48003,"POLYGON ((792219.493 1130103.506, 792219.283 1...",J,18036,Andrews
2,48005,"POLYGON ((1527772.708 998471.159, 1527771.700 ...",H,87322,Angelina


In [5]:
# Calculate population, population ratio over the county, county, TSA of each census tract
tract = gpd.read_file(os.path.join(data, 'original_data/demand_related/tl_2019_48_tract/tl_2019_48_tract.shp'))
tract = tract[['GEOID', 'geometry']]
tract['GEOID_County'] = tract.apply(lambda x:x['GEOID'][0:5], axis=1)

# Append TSA 
tract = tract.merge(county[['GEOID', 'TSA', 'County']], left_on='GEOID_County', right_on='GEOID')
tract['GEOID'] = tract['GEOID_x']
tract.drop(columns=['GEOID_x', 'GEOID_y'], inplace=True)

# Append census tract poulation 
tract = tract.merge(acs_pop, left_on='GEOID', right_on='Tract')
tract['County'] = tract['County_x']
tract.drop(columns=['County_x', 'County_y', 'GEO_ID', 'Tract'], inplace=True)

# Append county-level population and calculate the population ratio of each census tract over the county
tract = tract.merge(county_pop, left_on='GEOID_County', right_on='County')
tract['County'] = tract['County_x']
tract.drop(columns=['County_x', 'County_y'], inplace=True)
tract['pop'] = tract['pop_x']
tract['pop_ratio'] = tract['pop'] / tract['pop_y']
tract.drop(columns=['pop_x', 'pop_y', 'GEOID_County'], inplace=True)

tract.sort_values(by='GEOID', inplace=True, ignore_index=True)
tract = tract.to_crs(epsg=3081)

tract.to_file(os.path.join(data, 'reference_data', 'geographic_units', 'tract_reference.shp'))
tract.head(3)

/Users/jparkgeo/opt/anaconda3/envs/sa/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,geometry,TSA,GEOID,County,pop,pop_ratio
0,"POLYGON ((1405776.557 1097438.688, 1405779.438...",G,48001950100,Anderson,4844,0.083792
1,"POLYGON ((1392461.585 1072740.298, 1392447.163...",G,48001950401,Anderson,4838,0.083688
2,"POLYGON ((1379516.856 1075774.243, 1379519.198...",G,48001950402,Anderson,7511,0.129926


## Focus period

In [6]:
from_date = '07/01/2020'
to_date = '12/31/2021'

# start_date = datetime.datetime.strptime(from_date,  "%Y-%m-%d")
# end_date = datetime.datetime.strptime(to_date,  "%Y-%m-%d")
start_date = datetime.datetime.strptime(from_date,  "%m/%d/%Y")
end_date = datetime.datetime.strptime(to_date,  "%m/%d/%Y")

focus_date = []
delta = datetime.timedelta(days=1)
while start_date <= end_date:
    focus_date.append(start_date.strftime("%m/%d/%Y"))
    start_date += delta
    
# Add 06/30/2020 to the focus date to calculate daily . 
focus_date_ = focus_date.copy()
focus_date_.insert(0, '06/30/2020')

# Make a dictionary that has keys as target date and values as the date that should be averaged. 
focus_date_dict = {}
time_delta = [3, 2, 1, 0, -1, -2, -3]

for idx, date in enumerate(focus_date):
    temp_list = []
    for delta in time_delta:
        temp_list.append(
            str(
                (datetime.datetime.strptime(focus_date[idx], "%m/%d/%Y") - datetime.timedelta(days=delta)
                ).strftime("%m/%d/%Y"))
        )
        
    focus_date_dict[date] = temp_list
    
# Manually enter the dates that would have missing values
focus_date_dict['07/01/2020'] = ['07/01/2020', '07/02/2020', '07/03/2020', '07/04/2020']
focus_date_dict['07/02/2020'] = ['07/01/2020', '07/02/2020', '07/03/2020', '07/04/2020', '07/05/2020']
focus_date_dict['07/03/2020'] = ['07/01/2020', '07/02/2020', '07/03/2020', '07/04/2020', '07/05/2020', '07/06/2020']
focus_date_dict['12/29/2021'] = ['12/26/2021', '12/27/2021', '12/28/2021', '12/29/2021', '12/30/2021', '12/31/2021']
focus_date_dict['12/30/2021'] = ['12/27/2021', '12/28/2021', '12/29/2021', '12/30/2021', '12/31/2021']
focus_date_dict['12/31/2021'] = ['12/28/2021', '12/29/2021', '12/30/2021', '12/31/2021']
# focus_date_dict

## COVID-19 data

In [7]:
# Load daily confirmed case data
daily_case = pd.read_excel(os.path.join(data, 'original_data/demand_related/Texas COVID-19 New Confirmed Cases by County.xlsx'),
                           sheet_name='New Cases by County 2020',
                           header=2,
                           index_col=0)

daily_case = daily_case.loc['Anderson':'Zavala', focus_date]
daily_case.columns = daily_case.columns.astype(str)

# Change the name of columns
new_cols = [datetime.datetime.strptime(col, '%Y-%m-%d').strftime('%m/%d/%Y') for col in daily_case.columns]
daily_case.rename(columns={daily_case.columns[i]: new_cols[i] for i in range(len(new_cols))}, inplace=True)

# Remove negative value of daily case. 
# Negative value has replaced with 0
for col in daily_case.columns:
    daily_case[col] = daily_case[col].mask(daily_case[col] < 0, 0)

daily_case.to_csv(os.path.join(data, 'reference_data/covid_data/daily_covid_case.csv'))
daily_case.head(3)

,07/01/2020,07/02/2020,07/03/2020,07/04/2020,07/05/2020,07/06/2020,07/07/2020,07/08/2020,07/09/2020,07/10/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
County,,,,,,,,,,,,,,,,,,,,,
Anderson,4.0,8.0,4.0,26.0,2.0,4.0,9.0,8.0,3.0,15.0,...,1.0,6.0,2.0,7.0,3.0,7.0,4.0,13.0,4.0,4.0
Andrews,2.0,4.0,2.0,1.0,0.0,9.0,3.0,6.0,10.0,8.0,...,3.0,10.0,5.0,0.0,0.0,0.0,0.0,16.0,27.0,16.0
Angelina,8.0,63.0,17.0,11.0,2.0,8.0,18.0,3.0,23.0,1.0,...,1.0,7.0,6.0,6.0,7.0,9.0,9.0,11.0,7.0,23.0


In [12]:
# Loaded data is the accumulated death per county
covid_death = pd.read_excel(os.path.join(data, 'original_data/demand_related/Texas COVID-19 Fatality Count Data by County.xlsx'),
                             sheet_name='Fatalities by County 2020',
                             header=2,
                             index_col=0)

covid_death = covid_death.loc['Anderson':'Zavala', focus_date_]

# Calculate daily death from the accumulated value
daily_death = pd.DataFrame(index=covid_death.index, columns=focus_date_[1:])

for i, date in enumerate(focus_date_):
    if i != 0:
        daily_death[date] = covid_death[date] - covid_death[focus_date_[i -1]]

# Remove negative value of daily death. 
# Negative value has replaced with 0
for col in daily_death.columns:
    daily_death[col] = daily_death[col].mask(daily_death[col] < 0, 0)
        

daily_death.to_csv(os.path.join(data, 'reference_data/covid_data/daily_covid_death.csv'))
daily_death.head(3)

,07/01/2020,07/02/2020,07/03/2020,07/04/2020,07/05/2020,07/06/2020,07/07/2020,07/08/2020,07/09/2020,07/10/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
County,,,,,,,,,,,,,,,,,,,,,
Anderson,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
Andrews,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Angelina,0,1,0,1,0,2,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


## 7-day Averaged COVID-19 Data

In [27]:
ave_case = daily_case.copy(deep=True)
ave_death = daily_death.copy(deep=True)

for idx, val in focus_date_dict.items():
    ave_death[idx] = daily_death[val].mean(axis=1)
    ave_case[idx] = daily_case[val].mean(axis=1)
    
ave_case = ave_case.reset_index()
ave_death = ave_death.reset_index()

ave_case.to_csv(os.path.join(data, 'reference_data', 'covid_data', 'averaged_covid_case.csv'), index=False)
ave_death.to_csv(os.path.join(data, 'reference_data', 'covid_data', 'averaged_covid_death.csv'), index=False)

ave_case.head(3)

,County,07/01/2020,07/02/2020,07/03/2020,07/04/2020,07/05/2020,07/06/2020,07/07/2020,07/08/2020,07/09/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
0,Anderson,10.50,8.8,8.000000,8.142857,8.714286,8.000000,9.571429,14.285714,61.857143,...,3.857143,3.857143,4.285714,4.285714,6.000000,5.714286,6.000000,5.833333,6.4,6.25
1,Andrews,2.25,1.8,3.000000,3.000000,3.571429,4.428571,5.285714,5.428571,5.428571,...,3.428571,3.285714,3.285714,2.571429,4.428571,6.857143,8.428571,9.833333,11.8,14.75
2,Angelina,24.75,20.2,18.166667,18.142857,17.428571,11.714286,9.428571,11.285714,12.857143,...,5.000000,6.000000,6.428571,6.428571,7.857143,7.857143,10.285714,11.000000,11.8,12.50


In [18]:
ave_death.head(3)

,County,07/01/2020,07/02/2020,07/03/2020,07/04/2020,07/05/2020,07/06/2020,07/07/2020,07/08/2020,07/09/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
0,Anderson,0.25,0.2,0.166667,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.142857,0.285714,0.285714,0.428571,0.428571,0.285714,0.285714,0.333333,0.2,0.25
1,Andrews,0.00,0.2,0.166667,0.142857,0.142857,0.142857,0.142857,0.142857,0.000000,...,0.142857,0.142857,0.000000,0.142857,0.142857,0.142857,0.142857,0.166667,0.2,0.25
2,Angelina,0.50,0.4,0.666667,0.571429,0.571429,0.428571,0.571429,0.571429,0.571429,...,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.166667,0.2,0.25
